## Import Dependencies, Model, and Data

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import files
import numpy as np

In [2]:
# Read in data to pandas df
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Drop NAME and EIN Col
application_df = application_df.drop(columns=['NAME', 'EIN'])

application_df.info()
application_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Convert 'Status' col to object dtype
application_df['STATUS'] = application_df['STATUS'].astype('object')

# Define target and features
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Separate cat and cont cols
cat_cols = X.select_dtypes(include=['object'])
cont_cols = X.select_dtypes(include=['int64'])

In [4]:
# Encode cat_cols with get_dummies
cat_cols = pd.get_dummies(cat_cols)

# log skewed features in cont_cols
for col in cont_cols:
    if (cont_cols[col] > 0).all():  # Log transform only makes sense for positive values
        cont_cols[col] = np.log1p(cont_cols[col])

# Scale cont_cols with Standard Scaler
scaler = StandardScaler()
cont_cols = scaler.fit_transform(cont_cols)


In [5]:
# # Concatenate cat_cols and cont_cols
X = np.concatenate((cat_cols, cont_cols), axis=1).astype('int64')

# # Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# Define the model
model = Sequential()

# First hidden layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))

# Second hidden layer
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

In [7]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

In [8]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
644/644 [==============================] - 7s 4ms/step - loss: 0.5880 - accuracy: 0.7072 - val_loss: 0.5626 - val_accuracy: 0.7240 - lr: 0.0010
Epoch 2/100
644/644 [==============================] - 3s 4ms/step - loss: 0.5587 - accuracy: 0.7308 - val_loss: 0.5577 - val_accuracy: 0.7263 - lr: 0.0010
Epoch 3/100
644/644 [==============================] - 3s 4ms/step - loss: 0.5529 - accuracy: 0.7330 - val_loss: 0.5560 - val_accuracy: 0.7279 - lr: 0.0010
Epoch 4/100
644/644 [==============================] - 3s 4ms/step - loss: 0.5516 - accuracy: 0.7352 - val_loss: 0.5543 - val_accuracy: 0.7296 - lr: 0.0010
Epoch 5/100
644/644 [==============================] - 3s 5ms/step - loss: 0.5484 - accuracy: 0.7342 - val_loss: 0.5557 - val_accuracy: 0.7320 - lr: 0.0010
Epoch 6/100
644/644 [==============================] - 3s 4ms/step - loss: 0.5467 - accuracy: 0.7357 - val_loss: 0.5529 - val_accuracy: 0.7318 - lr: 0.0010
Epoch 7/100
644/644 [==============================] - 2s 4ms/st